In [1]:
import collections
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import math
import networkx as nx
import random
from IPython.display import clear_output

import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
from PIL import Image
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [2]:
def run_lru(cache_dict, cache_size, seq, start, box_width, hit_cost, miss_cost):   
    i = start
    remain_width = box_width
    while remain_width>0:
        if seq[i] in cache_dict.keys():
            remain_width=remain_width-hit_cost
            cache_dict.move_to_end(seq[i], last=False)
        else:
            remain_width=remain_width-miss_cost
            cache_dict[seq[i]]=True
            cache_dict.move_to_end(seq[i], last=False)
            if len(cache_dict.keys())>cache_size:
                cache_dict.popitem(last=True)
        i = i+1
        if i == len(seq):
            break
    return i # Where the box ends/The position of the next request in the sequence 

In [3]:
def get_vector(seq, pointer_position, window_size):
    vector = []
    if pointer_position == 0:
        for i in range(window_size + 4):
            vector.append(0.00)
    else:
        if pointer_position < window_size:
            window = seq[0:pointer_position]
        else:
            window = seq[pointer_position - window_size:pointer_position]
        frequency = {}
        distinct = {}
        segs_distinct = [0.00, 0.00, 0.00, 0.00] # chop the window into 4 segments 
        #count how many distinct page ids in each seg 
        
        i=0
        seg_id = 0
        while i < len(window):
            if window[i] in frequency.keys():
                frequency[window[i]] = frequency[window[i]] + 1.00
            else:
                frequency[window[i]] = 1.00
            distinct[window[i]] = True
            if (i+1) % (int(window_size / 4)) == 0:
                segs_distinct[seg_id] = len(distinct.keys())
                distinct = {}
                seg_id = seg_id + 1
            i = i + 1
        
        if len(distinct.keys()) > 0:
            segs_distinct[seg_id] = len(distinct.keys())
        
        # 1<=j<=w, the j-th variable of the vector is the frequency of the j-th most frequent page id in 
        # the window
        for v in frequency.values():
            vector.append(v)
        while len(vector) < window_size:
            vector.append(0)
        vector.sort(reverse=True)
        
        # We chop the w requests into four segments of length w/4. 
        # Count how many distinct ids in each segment, and put the countings in the last four variables.
        for v in segs_distinct:
            vector.append(v)
    return vector

In [4]:
class DQN(nn.Module):
    def __init__(self, window_size, num_of_box_kind):
        super().__init__()
        self.conv1 = nn.Conv1d(window_size+4, 512, 1)
        self.pool = nn.MaxPool1d(kernel_size=2)
        self.conv2 = nn.Conv1d(256, 128, 1)
        self.fc1 = nn.Linear(64, 32)
        self.fc2 = nn.Linear(32, num_of_box_kind)
        # print('.........'+str(num_of_box_kind))

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool(torch.transpose(x,1,2))
        x = self.conv2(torch.transpose(x,1,2))
        x = self.pool(torch.transpose(x,1,2))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return x

In [5]:
Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([],maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [6]:
# Maverick: I add a no_more_random. 
# In the final epochs of training, I can choose to ban the model from acting randomly.
# Guarantee the model converge.
def select_action(state, steps_done, EPS_START, EPS_END, 
                  EPS_DECAY, no_more_random, num_of_box_kinds):
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold or no_more_random:
        with torch.no_grad():
            # t.max(1) will return largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            return torch.argmax(policy_net(state)), steps_done
    else:
        return torch.tensor([[random.randrange(num_of_box_kinds)]], device=device, dtype=torch.long), steps_done

In [7]:
def optimize_model(memory, BATCH_SIZE, ALPHA, GAMMA, policy_net, optimizer):
    if len(memory) < BATCH_SIZE:
        return policy_net, optimizer
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    #print(state_batch.shape)
    state_action_values = policy_net(state_batch).gather(0, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    nsv=target_net(non_final_next_states).max(1)[0].detach()
    lst=[]
    for row in nsv:
        lst.append(torch.argmax(row)*1.00)
    next_state_values[non_final_mask] = torch.tensor(lst,device=device)
    # Compute the expected Q values
    
    ##############################################
    ## Maverick: I add learning rate ALPHA here ##
    ##############################################
    expected_state_action_values = ALPHA*((next_state_values * GAMMA) + reward_batch)

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()
    return policy_net, optimizer

In [9]:
f = open("seq-sort10k.ssv")
data = f.readline()
seq=data.split(' ')
print(seq[:10])
print(len(seq))

['0', '1000', '0', '1999', '1000', '1999', '1999', '0', '0', '0']
400961


In [10]:
BATCH_SIZE = 8
GAMMA = 0.3
EPS_START = 0.9
EPS_END = 0.00001
EPS_DECAY = 500
TARGET_UPDATE = 5
window_size=256
miss_cost = 100 ##################
# number_of_box_kinds = min(8,math.ceil(math.log2(miss_cost))) #############
number_of_box_kinds=8
NUMBER_OF_MODELS = 5 # Train several models, choose the best
num_episodes = 10
ALPHA = 0.8 # Learning rate #########################

In [11]:
features = []
for i in range(len(seq)):
    features.append(get_vector(seq, i, window_size))

In [15]:
best_result=[math.inf for _ in range(num_episodes)]
best_hist = [0 for _ in range(number_of_box_kinds)]
print(len(best_result))
best_policy_net = DQN(window_size,number_of_box_kinds).to(device)
best_target_net = DQN(window_size,number_of_box_kinds).to(device)
best_seeds=DQN(window_size,number_of_box_kinds).to(device)

for model in range(NUMBER_OF_MODELS):
    global_lru=collections.OrderedDict()
    steps_done = 0
    policy_net = DQN(window_size,number_of_box_kinds).to(device)
    target_net = DQN(window_size,number_of_box_kinds).to(device)
    target_net.load_state_dict(policy_net.state_dict())
    target_net.eval()
    seeds=DQN(window_size,number_of_box_kinds).to(device)
    seeds.load_state_dict(policy_net.state_dict())

    optimizer = optim.RMSprop(policy_net.parameters())
    memory = ReplayMemory(10000)
    result = []
    hist = [0 for _ in range(number_of_box_kinds)] 

    for i_episode in range(num_episodes):
        for xx in range(number_of_box_kinds):
            hist[xx]=0
        pointer = 0
        impact = 0
        while pointer < len(seq):
            startpointer = pointer
            state = features[pointer]
            state = torch.tensor([[state]], device=device)
            state = torch.transpose(state,1,2)
            action, steps_done = select_action(state,steps_done,EPS_START,EPS_END,EPS_DECAY,
                                               (i_episode+1)/num_episodes>0.9,number_of_box_kinds)
            box_id=action
            # print(box_id)
            hist[box_id]=hist[box_id]+1
            cache_size=2**box_id
            box_width = miss_cost*cache_size
            
            # Compartmentalization
            # Load top pages from LRU stack.
            mycache = collections.OrderedDict()
            for pid in global_lru.keys():
                mycache[pid]=True
                mycache.move_to_end(pid, last=True)###########
                if len(mycache)==cache_size:
                    break

            action = torch.tensor([[[action]]], device=device) # make it a tensor
            pointer =run_lru(mycache, cache_size, seq, pointer, box_width, 1, miss_cost)
            endpointer = pointer
            
            # Update global stack
            for x in range(startpointer, endpointer):
                if seq[x] in global_lru.keys():
                    global_lru.move_to_end(seq[x], last=False)
                else:
                    global_lru[seq[x]]=True
                    global_lru.move_to_end(seq[x], last=False)
                    
            # print(mi)
            area = 3*miss_cost*cache_size*cache_size
            impact=impact+area
            reward = torch.tensor([[[-area]]], device=device)

            if pointer < len(seq):
                next_state = features[pointer]
                next_state = torch.tensor([[next_state]], device=device)
                next_state = torch.transpose(next_state,1,2)
            else:
                next_state = None

            # Store the transition in memory
            memory.push(state, action, next_state, reward)

            # Move to the next state
            state = next_state

            # Perform one step of the optimization (on the policy network)
            policy_net, optimizer=optimize_model(memory, BATCH_SIZE, ALPHA, GAMMA, policy_net, optimizer)

        # Update the target network, copying all weights and biases in DQN
        clear_output()
        print(best_result[-1])
        print('MODEL-'+str(model))
        print('epoch='+str(i_episode)+'..........impact='+str(impact))
        result.append(impact.item())


        if i_episode % TARGET_UPDATE == 0:
            target_net.load_state_dict(policy_net.state_dict())

    print('Complete')
    if result[-1]<best_result[-1]:
        best_seeds.load_state_dict(seeds.state_dict())
        for idx in range(len(result)):
            best_result[idx]=result[idx]
        for idx in range(len(hist)):
            best_hist[idx]=hist[idx]
            
        # store the best for further use
        best_policy_net.load_state_dict(policy_net.state_dict())
        best_target_net.load_state_dict(target_net.state_dict())
        
print(best_hist)

In [16]:
torch.save({'seeds':best_seeds.state_dict()}, './best_seeds.t7')

In [15]:

global_lru=collections.OrderedDict()
steps_done = 0
bs = torch.load('best_seeds.t7')
policy_net = DQN(window_size,number_of_box_kinds).to(device)
policy_net.load_state_dict(bs['seeds'])
target_net = DQN(window_size,number_of_box_kinds).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.RMSprop(policy_net.parameters())
memory = ReplayMemory(10000)
result = []

for i_episode in range(num_episodes):
    pointer = 0
    impact = 0
    while pointer < len(seq):
        startpointer = pointer
        state = features[pointer]
        state = torch.tensor([[state]], device=device)
        state = torch.transpose(state,1,2)
        action, steps_done = select_action(state,steps_done,EPS_START,EPS_END,EPS_DECAY,
                                           (i_episode+1)/num_episodes>0.9,number_of_box_kinds)
        box_id=action
        cache_size=2**box_id
        box_width = miss_cost*cache_size

        # Compartmentalization
        # Load top pages from LRU stack.
        mycache = collections.OrderedDict()
        for pid in global_lru.keys():
            mycache[pid]=True
            mycache.move_to_end(pid, last=True)###########
            if len(mycache)==cache_size:
                break

        action = torch.tensor([[[action]]], device=device) # make it a tensor
        pointer =run_lru(mycache, cache_size, seq, pointer, box_width, 1, miss_cost)
        endpointer = pointer

        # Update global stack
        for x in range(startpointer, endpointer):
            if seq[x] in global_lru.keys():
                global_lru.move_to_end(seq[x], last=False)
            else:
                global_lru[seq[x]]=True
                global_lru.move_to_end(seq[x], last=False)

        # print(mi)
        area = 3*miss_cost*cache_size*cache_size
        impact=impact+area
        reward = torch.tensor([[[-area]]], device=device)

        if pointer < len(seq):
            next_state = features[pointer]
            next_state = torch.tensor([[next_state]], device=device)
            next_state = torch.transpose(next_state,1,2)
        else:
            next_state = None

        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the policy network)
        policy_net, optimizer=optimize_model(memory, BATCH_SIZE, ALPHA, GAMMA, policy_net, optimizer)

    # Update the target network, copying all weights and biases in DQN
    #clear_output()
    #print(best_result[-1])
    print('epoch='+str(i_episode)+'..........impact='+str(impact))
    result.append(impact.item())


    if i_episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())

print('Complete')

epoch=0..........impact=tensor([[333387000]], device='cuda:0')
epoch=1..........impact=tensor([[45029100]], device='cuda:0')
epoch=2..........impact=tensor(34104000, device='cuda:0')


KeyboardInterrupt: 

In [ ]:
while True:
    clear_output()
    print('hello')